In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

## What is RAG (retrieval augmented generation)?
Basically, shoving lot of extra information in the prompt.

In [46]:
# Example 

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(
    prompt_template.format(
        context="Ajit has two sisters, Preeti and Sweta. Ajit is male.",
        question="Who is Preeti's bother?"         
    ))

print(response)

content="Ajit is Preeti's brother. He is also the brother of Sweta. There is no other information about any other brother of Preeti." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 119, 'total_tokens': 152, 'completion_time': 0.141294585, 'prompt_time': 0.016818076, 'queue_time': 0.050254424, 'total_time': 0.158112661}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--e932601c-af80-4984-a4fb-43f85672e977-0' usage_metadata={'input_tokens': 119, 'output_tokens': 33, 'total_tokens': 152}


In [47]:
print(response.content)

Ajit is Preeti's brother. He is also the brother of Sweta. There is no other information about any other brother of Preeti.


>RAG is all about cleverly pushing is as much information in the context with minimum possible tokens

In [48]:
# little non-trival RAG example

# grab a novel
file = open("indianTales.txt")
text = file.read()

In [49]:
# lets try to talk with this book

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Prepare your prompt
prompt_template = """You are a novel reader. You are given collection of stories:
{collection_of_stories}
You are tasked to make a list of story titles in this collection. Write a short summary for each story in Hindi Language. Skip the story from the list, if the story is not provided in the text. 
"""

response = model.invoke(prompt_template.format(collection_of_stories = text[:len(text)//20]))

print(response)

content='यहाँ संग्रह में शीर्षकों की सूची दी गई है:\n\n1. **शेर और बगुला**: यह कहानी एक शेर और एक बगुले के बारे में है, जो एक दूसरे से मिलते हैं और एक दूसरे की मदद करते हैं। बगुला शेर के गले में फंसी हुई हड्डी को निकालता है, लेकिन शेर उसका शुक्रिया नहीं अदा करता है।\n2. **राजा के पुत्र ने कैसे राजकुमारी लबाम को जीता**: यह कहानी एक राजकुमार के बारे में है, जो अपनी माँ की बात मानकर एक जंगल में नहीं जाता है, लेकिन बाद में वह जंगल में जाता है और राजकुमारी लबाम को ढूंढने का फैसला करता है। वह कई चुनौतियों का सामना करता है और अंत में राजकुमारी लबाम से मिलता है।\n3. **लंबिकिन**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।\n4. **पंचकिन**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।\n5. **टूटा हुआ बर्तन**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।\n6. **जादुई बांसुरी**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।\n7. **क्रूर बगुला को मात देना**: इस कहानी का विवरण नहीं दिय

In [50]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

यहाँ संग्रह में शीर्षकों की सूची दी गई है:

1. **शेर और बगुला**: यह कहानी एक शेर और एक बगुले के बारे में है, जो एक दूसरे से मिलते हैं और एक दूसरे की मदद करते हैं। बगुला शेर के गले में फंसी हुई हड्डी को निकालता है, लेकिन शेर उसका शुक्रिया नहीं अदा करता है।
2. **राजा के पुत्र ने कैसे राजकुमारी लबाम को जीता**: यह कहानी एक राजकुमार के बारे में है, जो अपनी माँ की बात मानकर एक जंगल में नहीं जाता है, लेकिन बाद में वह जंगल में जाता है और राजकुमारी लबाम को ढूंढने का फैसला करता है। वह कई चुनौतियों का सामना करता है और अंत में राजकुमारी लबाम से मिलता है।
3. **लंबिकिन**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।
4. **पंचकिन**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।
5. **टूटा हुआ बर्तन**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।
6. **जादुई बांसुरी**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।
7. **क्रूर बगुला को मात देना**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।
8. **प्यार करने वाली लैली**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।
9. **बाघ, ब्राह्मण और शिकारी**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।
10. **भविष्यवाणी करने वाले का पुत्र**: इस कहानी का विवरण नहीं दिया गया है, इसलिए मैं इसे सूची से बाहर रख रहा हूँ।

इन कहानियों के अलावा, सूची में कई अन्य कहानियाँ हैं, लेकिन उनके विवरण नहीं दिए गए हैं, इसलिए मैं उन्हें सूची से बाहर रख रहा हूँ।

### Problem: Too much raw information in the context makes the prompt too long.
- Costly
- adds noise
### Solution: Use RAG
https://python.langchain.com/docs/tutorials/rag/
### To understand RAG, we need to understand Semantic Search
https://python.langchain.com/docs/tutorials/retrievers/

In [51]:
# load text using RELEVANT loader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

In [52]:
# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

Split given book into 563 sub-documents.


### Embedding

In [53]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
# (CLASSROOM DISCUSSION: What are vector stores? What do we make them?)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


In [54]:
document_ids

['7b4044c7-a9ac-4767-b9c8-5b7b1013c421',
 'b87037ca-d4c6-4af5-83db-572b41435bac',
 '8f084b7a-a5c6-4044-a2ed-5f194b07558e',
 '9d34fc1a-683c-4545-ae25-b9796aa9c627',
 'ec714037-9fbd-4772-ad0b-a613d6ab9c44',
 '167d6e7c-2f5b-4232-9304-981e528331e4',
 'afc739ff-6170-4424-aa30-e9bb55dc6bdc',
 '1b74ad8d-a3e6-43b4-8167-41f693a081a7',
 '1c8bcd9f-4134-4c33-bc40-4043f428e3d7',
 '193f9acd-f287-4da7-a109-0576877d376e',
 'bde262ab-15df-40e6-a875-2483f5909751',
 'ade1e4a3-e927-4b95-91b3-78bf3c11f18c',
 '93b38069-0bb1-419f-b06e-50d2303dd52b',
 '19f31730-4474-4561-a838-9eb173e0b96e',
 '85ab4941-b285-46cf-a496-eee30ccc65f8',
 '51f659fc-dd33-4602-a068-0e8dadce8ea6',
 '2b98733b-8d35-4e2d-9dbb-8884913d3939',
 '6d13849d-b2d8-4c79-8cd3-3cf13a0dbd0e',
 '4149ed32-979e-4a18-ab19-bd8e2f779406',
 '63e0ddff-1cf7-47b2-bea0-10069212f7b1',
 '3f2d36f0-066d-454f-a545-6167b23aed17',
 '1cbdcd7c-16d6-410c-b517-d164ea2e3540',
 '1f276318-4a56-4c7a-af63-420efedaab88',
 '172132d6-5cc0-43d6-8672-474795cc3ff4',
 'f9327302-078a-

In [55]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

In [56]:
search_results

[(Document(id='24e89f2c-53dd-4335-85d6-259ea65fb697', metadata={'source': 'indianTales.txt', 'start_index': 341149}, page_content="In the following Notes I give, as on the two previous occasions, the\n_source_ whence I derived the tale, then _parallels_, and finally\n_remarks_. For Indian _parallels_ I have been able to refer to Major\nTemple's remarkable Analysis of Indian Folk-tale incidents at the end\nof _Wideawake Stories_ (pp. 386-436), for European ones to my\nalphabetical List of Incidents, with bibliographical references, in\n_Transactions of Folk-Lore Congress_, 1892, pp. 87-98. My _remarks_\nhave been mainly devoted to tracing the relation between the Indian\nand the European tales, with the object of showing that the latter\nhave been derived from the former. I have, however, to some extent\nhandicapped myself, as I have avoided giving again the Indian versions\nof stories already given in _English Fairy Tales_ or _Celtic Fairy\nTales_.\n\n\nI. THE LION AND THE CRANE."),
  

### What is RAG?
Retrieve using semantic search and dump the similar chunks in the context of the prompt.
LLM sees the question and retrieved docs in its prompt and generates tokens accordingly.

In [57]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

In [58]:
doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

In the following Notes I give, as on the two previous occasions, the
_source_ whence I derived the tale, then _parallels_, and finally
_remarks_. For Indian _parallels_ I have been able to refer to Major
Temple's remarkable Analysis of Indian Folk-tale incidents at the end
of _Wideawake Stories_ (pp. 386-436), for European ones to my
alphabetical List of Incidents, with bibliographical references, in
_Transactions of Folk-Lore Congress_, 1892, pp. 87-98. My _remarks_
have been mainly devoted to tracing the relation between the Indian
and the European tales, with the object of showing that the latter
have been derived from the former. I have, however, to some extent
handicapped myself, as I have avoided giving again the Indian versions
of stories already given in _English Fairy Tales_ or _Celtic Fairy
Tales_.


I. THE LION AND THE CRANE.


mouth struck one end of the bone with his beak. Whereupon the bone
dropped and fell out. As soon as he had caused the bone to fall, he
got out of the

In [59]:
# make the LLM read see the prompt, and analyse the retrieved document, and generate response

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
# (CLASSROOM DISCUSSION: What are vector stores? What do we make them?)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)



from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))

In [60]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

The lion is a main character in the story "The Lion and the Crane", where it gets a bone stuck in its mouth and is helped by a crane. The lion shows no gratitude towards the crane and instead threatens it, saying it's lucky to be alive after being between its teeth. The lion is also said to represent Devadatta the Traitor in the story as told by Gautama the Buddha.

# RAG Summary

In [61]:
# RAG summary

# Read a doc
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

# embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create a vector store
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


Split given book into 563 sub-documents.


In [62]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

In the following Notes I give, as on the two previous occasions, the
_source_ whence I derived the tale, then _parallels_, and finally
_remarks_. For Indian _parallels_ I have been able to refer to Major
Temple's remarkable Analysis of Indian Folk-tale incidents at the end
of _Wideawake Stories_ (pp. 386-436), for European ones to my
alphabetical List of Incidents, with bibliographical references, in
_Transactions of Folk-Lore Congress_, 1892, pp. 87-98. My _remarks_
have been mainly devoted to tracing the relation between the Indian
and the European tales, with the object of showing that the latter
have been derived from the former. I have, however, to some extent
handicapped myself, as I have avoided giving again the Indian versions
of stories already given in _English Fairy Tales_ or _Celtic Fairy
Tales_.


I. THE LION AND THE CRANE.


mouth struck one end of the bone with his beak. Whereupon the bone
dropped and fell out. As soon as he had caused the bone to fall, he
got out of the

In [63]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))


from IPython.display import Markdown
Markdown(response.content)

The Lion is a main character in the story "The Lion and the Crane", where it gets a bone stuck in its mouth and is helped by a crane. The Lion shows no gratitude towards the crane and even threatens it, indicating its ungrateful nature. The story is also referenced as a Jataka, with the Lion representing Devadatta the Traitor, and the crane representing Gautama the Buddha.